# Prediction using NN

In [8]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

ImportError: cannot import name 'cast' from partially initialized module 'keras.src.backend' (most likely due to a circular import) (/home/benson/.local/lib/python3.10/site-packages/keras/src/backend/__init__.py)

## Load Dataset

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## EDA & Cleaning

In [3]:
# Drop the columns that are not useful for the model - PassengerId, Name, Ticket
train_df = train_df.drop(['PassengerId','Name','Ticket'], axis=1)

train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S
5,0,3,male,NaN,0,0,8.4583,NaN,Q
6,0,1,male,54.0,0,0,51.8625,E46,S
7,0,3,male,2.0,3,1,21.0750,NaN,S
8,1,3,female,27.0,0,2,11.1333,NaN,S
9,1,2,female,14.0,1,0,30.0708,NaN,C


In [5]:
# !pip install ydata_profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(train_df, title='Train df Automated EDA Report', explorative=True)

# show in notebook
profile.to_notebook_iframe()

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 464.0 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 459.1 kB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 426.7 kB/s eta 0:00:000:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 419.1 kB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.14.6
    Uninstalling pydantic_core-2.14.6:
      Successfully uninstalled pydantic_core-2.14.6
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfully uninstalled numpy-1.26.3
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency reso

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# check for missing values
train_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [11]:
train_df.shape

(891, 9)

In [13]:
# drop empty embarked rows
train_df.dropna(subset=['Embarked'])

# fill missing age values with the mean
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())

# check for missing values
train_df.isnull().sum()

/tmp/ipykernel_30972/2346064574.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())


Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      0
dtype: int64

In [21]:
# Transform the categorical data into numerical data
train_df = pd.get_dummies(train_df, columns=['Sex', 'Cabin', 'Embarked'], dtype='int64')

train_df.head(10)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A10,Cabin_A14,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.000000,1,0,7.2500,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,38.000000,1,0,71.2833,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,3,26.000000,0,0,7.9250,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,35.000000,1,0,53.1000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,3,35.000000,0,0,8.0500,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,3,29.642093,0,0,8.4583,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
6,0,1,54.000000,0,0,51.8625,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,3,2.000000,3,1,21.0750,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,1,3,27.000000,0,2,11.1333,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,1,2,14.000000,1,0,30.0708,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Feature Engineering

## Model training

In [26]:
train_data = train_df.drop('Survived', axis=1)
train_labels = train_df['Survived']

In [28]:
# make a nerual network model
%pip install keras
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(28,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=10, batch_size=32)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
# evaluate model
model.evaluate(train_data, train_labels)

## Model Evaluation

In [22]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [23]:
# Preprocess the test data 
test_df = test_df.drop(['PassengerId','Name','Ticket'], axis=1)

In [25]:
# Transform the categorical data into numerical data
test_df = pd.get_dummies(test_df, columns=['Sex', 'Cabin', 'Embarked'], dtype='int64')

test_df.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A11,Cabin_A18,Cabin_A21,...,Cabin_F E46,Cabin_F E57,Cabin_F G63,Cabin_F2,Cabin_F33,Cabin_F4,Cabin_G6,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
test_data = test_df.drop('Survived', axis=1)
train_labels = test_df['Survived']

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)

print('Test accuracy:', test_acc)